In [1]:
import json
from cohere import ClientV2
from openai import OpenAI
import time


In [ ]:
LANGUAGE = "javascript"
PACKAGE_MANAGER = "NPM"

# LANGUAGE = "python"
# PACKAGE_MANAGER = "PYPI"



In [2]:
with open(f"../../results/hallucinations_{LANGUAGE}.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    
data = dict(sorted(data.items(), key=lambda x: (-x[1], x[0])))
len(data)

46

In [ ]:
SECRET = ""

In [7]:

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=SECRET,
)


def ask_llm(library_name: str):
    output = client.chat.completions.create(
		model="deepseek/deepseek-r1-0528-qwen3-8b:online",
		#":online" here means it uses google search.
		messages=[
					{
						"role": "system",
						"content": f"You are a helpful assistant that given a {LANGUAGE} library name, determines whether the library is a real {LANGUAGE} library that you can find in {PACKAGE_MANAGER}. Use google search to see if library exists. You will answer with either \"YES\" if it's a real {LANGUAGE} library and \"NO\" if it isn't. No other words."
					},
					{
						"role": "user",
						"content": library_name
					}
				],
	)  
    return output.choices[0].message.content

In [8]:
ask_llm("colyseus")

'YES [github.com/colyseus/colyseus](https://github.com/colyseus/colyseus), [npmjs.com/colyseus](https://www.npmjs.com/package/colyseus)'

In [12]:
hallucinated = {}
not_hallucinated = []

In [13]:
for name in data.keys():
    if name in not_hallucinated or name in hallucinated:
        continue
    
    response = ask_llm(name)
    print(response)
    if "YES" in response:
        not_hallucinated.append(name)
    elif "NO" in response:
        print(f"{name} is NOT a real library, occurrence: {data[name]}")
        hallucinated[name] = data[name]
    else:
        print(f"{name} yielded in neither no or yes")
        hallucinated[name] = data[name]
        
    # time.sleep(60/40)

NO
Penduel is NOT a real library, occurrence: 15
[github.com/keycloak/keycloak](https://github.com/keycloak/keycloak) [npmjs.com/package/keycloak-js](https://www.npmjs.com/package/keycloak-js) [keycloak.org](https://www.keycloak.org/)[medium.com](https://medium.com/)[github.com](https://github.com/keycloak/keycloak-client)
your-keycloak-library yielded in neither no or yes
NO
${newPackagePath} is NOT a real library, occurrence: 4
NO
${oldPackagePath} is NOT a real library, occurrence: 4
[github.com/XinFinOrg/MyContract](https://github.com/XinFinOrg/MyContract)
MyContract yielded in neither no or yes
NO
Shares is NOT a real library, occurrence: 3
Based on web search results, "@colyseus/nomination" is not explicitly referenced in the provided information, such as the official Colyseus documentation or NPM package listings. It is also not listed among the known related packages like "@colyseus/social", "@colyseus/monitor", or similar tools. Without evidence confirming its existence, the a

In [14]:
with open(f"../../results/hallucinations_{LANGUAGE}_cleaned_with_ai.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(hallucinated, indent=2))

In [15]:
with open(f"../../results/hallucinations_{LANGUAGE}_cleaned_with_ai.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    x = 0
    for a in data:
        if data[a] == 1:
            x += 1
    print(x)

27
